Analizador léxico
Edgar Fernando Espinosa Torres

In [ ]:
import csv

# Normalizar las claves y valores de los diccionarios a minúsculas
def normalizar_tabla(tabla):
    tabla_normalizada = {}
    for clave, valor in tabla.items():
        # Aquí, si el valor es un guión y no es un encabezado (estado), se cambia a "error".
        tabla_normalizada[int(clave)] = {str(k).lower(): ('error' if v == '-' else str(v).lower() if isinstance(v, str) else int(v))
                                         for k, v in valor.items()}
    return tabla_normalizada

# Leer la tabla de transiciones desde un archivo CSV y normalizarla
def leer_tabla_transiciones(ruta_archivo):
    with open(ruta_archivo, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader((line.lower() for line in csvfile))
        tabla_transiciones = {}
        for fila in reader:
            estado = int(fila['estado'])
            transiciones = {clave: (int(valor) if valor.isdigit() else valor) for clave, valor in fila.items() if clave != 'estado'}
            tabla_transiciones[estado] = transiciones
        return normalizar_tabla(tabla_transiciones)

# Alfabeto
alfabeto = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '.', 'e', '+', '-'} # Suponiendo que '-' sigue siendo parte del alfabeto

def caracter_a_entrada(caracter):
    if caracter.isdigit():
        return 'digito'
    elif caracter == '':
        return 'fdc'
    elif caracter == '-': # Aquí manejamos el '-' como un error fuera de los encabezados
        return 'error'
    elif caracter.lower() in alfabeto:
        return caracter.lower()
    else:
        return 'error'

def analizar_cadena(cadena, tabla_transiciones):
    estado = 1
    i = 0
    secuencia = ""
    while estado != 'aceptar' and estado != 'error':
        if i < len(cadena):
            simbolo = cadena[i]
        else:
            simbolo = ''  # Marca de fin de cadena (end of string)
        entrada = caracter_a_entrada(simbolo)
        proximo_estado = tabla_transiciones[estado].get(entrada, 'error')
        secuencia += f"Estado {estado}, {entrada} ---> Estado {proximo_estado}\n"
        if proximo_estado == 'error':
            return False, secuencia + "No se acepta la cadena."
        estado = proximo_estado
        i += 1
    return estado == 'aceptar', secuencia + "Sí se acepta la cadena."

# Consola para el usuario
print("Analizador Léxico. Escribe 'salir' para terminar.")
ruta_archivo = input("Introduce la ruta del archivo CSV con la tabla de transiciones: ")

# Leer la tabla de transiciones del archivo CSV
tabla_transiciones = leer_tabla_transiciones(ruta_archivo)

while True:
    cadena = input("Introduce una cadena para analizar: ").lower()
    if cadena == 'salir':
        break
    if not set(cadena).issubset(alfabeto):
        print("La cadena contiene caracteres que no están en el alfabeto del autómata.")
        continue
    resultado, secuencia = analizar_cadena(cadena, tabla_transiciones)
    print(secuencia)